In [1]:
import numpy as np
import random

In [10]:
#sample data, change later

num_universes = 50
num_variables = 6
max_iterations = 1000
P_demand = 32235000

lower_bound = np.array([50, 50, 20, 20, 200, 200])   # Lower bounds for each DER
upper_bound = np.array([100, 100, 50, 50, 400, 400]) # Upper bounds for each DER

alpha = np.array([0.001, 0.001, 0.002, 0.002, 0.005, 0.005])
beta = np.array([20, 20, 15, 15, 40, 40])
gamma = np.array([1000, 1000, 500, 500, 2000, 2000])


In [11]:
def fitness_function_with_penalty(universe, P_demand, penalty_factor=10000):
    P_total = np.sum(universe)

    original_cost = np.sum(alpha * universe**2 + beta * universe + gamma)

    penalty = penalty_factor * max(0, P_demand - P_total)

    penalized_cost = original_cost + penalty

    return penalized_cost


In [12]:
def initialize_universes(num_universes, num_variables, lower_bound, upper_bound):
    return np.random.uniform(low=lower_bound, high=upper_bound, size=(num_universes, num_variables))

universes = initialize_universes(num_universes, num_variables, lower_bound, upper_bound)


In [13]:
fitness = np.array([fitness_function_with_penalty(universe, P_demand) for universe in universes])


In [14]:
def white_black_hole_mechanism(universes, fitness):
    sorted_indices = np.argsort(fitness)
    best_universe = universes[sorted_indices[0]]
    worst_universe = universes[sorted_indices[-1]]

    for i in range(num_universes):
        if i != sorted_indices[0]:
            universes[i] += np.random.uniform(0, 1) * (best_universe - universes[i])
            universes[i] -= np.random.uniform(0, 1) * (universes[i] - worst_universe)

    return universes


In [15]:
def wormhole_mechanism(universes, iteration, max_iterations, lower_bound, upper_bound):
    WEP = 0.2 + iteration * (1.0 - 0.2) / max_iterations
    TDR = 1 - (iteration / max_iterations)**6

    for i in range(num_universes):
        if np.random.uniform(0, 1) < WEP:
            universes[i] += TDR * np.random.uniform(-1, 1, size=num_variables) * (upper_bound - lower_bound)
            universes[i] = np.clip(universes[i], lower_bound, upper_bound)

    return universes


In [16]:
best_solutions = []

for iteration in range(max_iterations):
    universes = white_black_hole_mechanism(universes, fitness)

    universes = wormhole_mechanism(universes, iteration, max_iterations, lower_bound, upper_bound)

    fitness = np.array([fitness_function_with_penalty(universe, P_demand) for universe in universes])

    best_fitness_idx = np.argmin(fitness)
    best_solutions.append(universes[best_fitness_idx])


In [17]:
best_universe = min(best_solutions, key=lambda u: fitness_function_with_penalty(u, P_demand))
min_cost = fitness_function_with_penalty(best_universe, P_demand)

print("Best solution (power outputs):", best_universe)
print("Minimum generation cost:", min_cost)


Best solution (power outputs): [ 97.83484898  99.00644683  44.38405996  24.62375445 391.24912212
 395.6740149 ]
Minimum generation cost: 322339517299.0917
